In [1]:
# import required libraries
import rioxarray as rxr
import numpy as np
from rasterio.windows import Window
import xarray as xr
from glob import glob
import random
from natsort import natsorted
from scipy import stats
import hvplot.xarray

In [2]:
# functions for normalization and converting to unsigned integer 8

def liner_percentile(x):
    a = 0
    b = 1
    c = np.percentile(x,2)
    d = np.percentile(x,98)
    f = (((x - c)*((b-a))/(d-c))) + a
    f = np.clip(f,0,1)
    f = (f*255).astype(np.uint8)
    return f

def minmax(x):
    a = 0
    b = 1
    c = x.min()
    d = x.max()
    f = (((x - c)*((b-a))/(d-c))) + a
    f = np.clip(f,0,1)
    f = (f*255).astype(np.uint8)
    return f

In [3]:
# define image path
image_path = glob('D:/DeepLearning/Projects/Arecanut_segmentation/Data/image/'+'*.tif')
mask_path = glob('D:/DeepLearning/Projects/Arecanut_segmentation/Data/mask/'+'*.tif')
total = 0
for z,(im,msk) in enumerate(zip(image_path,mask_path)):

    # define output path
    output_img_directory = 'D:/DeepLearning/Projects/Arecanut_segmentation/Data/chips/images/'
    output_mask_directory = 'D:/DeepLearning/Projects/Arecanut_segmentation/Data/chips/masks/'

    # set width and height of patch
    patch_width = 128
    patch_height = 128


    # reading image as xarray and creating patches using rasterio window module
    with rxr.open_rasterio(im,chunks={'x':640,'y':640}) as src, rxr.open_rasterio(msk,chunks={'x':640,'y':640}) as mask:

        # applying normalization functions
        src = xr.apply_ufunc(liner_percentile,src.isel(band=[2,1,0]).compute())
        width = (src.rio.width//patch_width)*patch_width
        height = (src.rio.height//patch_height)*patch_height

        total_images = (width*height)//(patch_width*patch_height)
        total += total_images

        for i in range(0, width, patch_width):
            for j in range(0, height, patch_height):
                window = Window(i, j, patch_width, patch_height)

                # Read the data from the raster for the current patch
                img_patch = src.rio.isel_window(window=window).rio.write_crs("EPSG:32643")
                mask_patch = mask.rio.isel_window(window=window).rio.write_crs("EPSG:32643")

                # Define the output filename based on the patch's position
                output_img_filename = f"{output_img_directory}/patch{z}_{i}_{j}.tif"
                output_mask_filename = f"{output_mask_directory}/patch{z}_{i}_{j}.tif"
                m = stats.mode(np.unique(mask_patch))
                if len(np.unique(mask_patch))==1:
                    # print(f'skiping the patch{z}_{i}_{j}')
                    continue
                else:
                    # write the raster as geotif
                  img_patch.rio.to_raster(output_img_filename)
                  mask_patch.rio.to_raster(output_mask_filename)

    # Close the source raster file when done
    src.close()
print(f'total no of images:{total}')

total no of images:1118


In [4]:
images = natsorted(glob('D:/DeepLearning/Projects/Arecanut_segmentation/Data/chips/images/'+'*.tif'))
masks =  natsorted(glob('D:/DeepLearning/Projects/Arecanut_segmentation/Data/chips/masks/'+'*.tif'))

In [5]:
images_aerators = len(images)
total = total
percnetage = (images_aerators/total)*100
print(f'total no of image:{total} \n'
      f'images with only aerators:{images_aerators}\n'
      f'percentage of images with aerators:{percnetage}')

total no of image:1118 
images with only aerators:825
percentage of images with aerators:73.79248658318426


In [6]:
i = random.randint(1, len(images))
image = images[i]
label = masks[i]
image = rxr.open_rasterio(image)
label = rxr.open_rasterio(label)
imageplot = image.hvplot.rgb(bands='band',x='x',y='y',height=500,width=500,title='Image')
labelplot = label.isel(band=0).hvplot.image(x='x',y='y',height=500,width=500,title='Label',colorbar=False)
imageplot+labelplot

:Layout
   .RGB.I   :RGB   [x,y]   (R,G,B)
   .Image.I :Image   [x,y]   (value)